# Central Exercise 6 - Superposition Principle

This script plots streamlines and potential lines for the superposition of a source-/sink-flow and a potential-vortex flow.

## Preparation

To prepare the plot, several helper functions have to be defined. This is done in the following cell.
First, the two libraries numpy and matplotlib.pyplot have to be imported in order to user their functionality. Note, these libraries are helpful in many scenarios, when data should be plotted. Next, helper classes that provide the complex velocity for the source-/sink-flow and the parallel flow are implemented. Finally, a function providing the meshgrid for the calculations is implemented.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define important colors
TUMOrange = "#E37222"
TUMBlue = "#3070b3"

# Define helper classes to get the complex velocity of elementary flows
class PotentialVortexFlow:

    def __init__(self, gamma: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the vortex flow class. Stores relevant quantities as class members.

        Args:
            gamma (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.gamma = gamma
        self.factor = self.gamma / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the vortex flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z_translated = z - self.translation_x - 1.0j * self.translation_y
        return - 1.0j * self.factor / z_translated

class SourceFlow:

    def __init__(self, source_strength: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            source_strength (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.source_strength = source_strength
        self.factor = self.source_strength / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z_translated = z - self.translation_x - 1.0j * self.translation_y
        return self.factor / z_translated
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X_meshgrid, Y_meshgrid = np.meshgrid(x_linspace, y_linspace)
    Z_meshgrid = X_meshgrid + 1.0j * Y_meshgrid
    return X_meshgrid, Y_meshgrid, Z_meshgrid

## Definition of domain

Here, relevant information to describe the domain is provided. It is used to generate the meshgrid for the calculations.

In [ ]:
x_min = -4.0
x_max = 4.0
y_min = -4.0
y_max = 4.0
resolution_per_unit_length = 20
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X_meshgrid, Y_meshgrid, Z_meshgrid = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))

## Calculation of the complex velocity

Initializes the relevant source, sink and potential vortex objects to calculate the complex velocity.

In [ ]:
source_strength = -1.0
gamma = -1.0

source = SourceFlow(source_strength, 0.0, 0.0)
vortex_flow = PotentialVortexFlow(gamma, 0.0, 0.0)

complex_velocity = source.get_complex_velocity(Z_meshgrid) + vortex_flow.get_complex_velocity(Z_meshgrid)
u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

## Plotting of the results

Plot the streamlines and potential lines.
Streamlines are plotted in TUMBlue.
Potential lines are plotted in TUMOrange.

In [ ]:
fig, ax = plt.subplots()

# Plot the streamlines.

# The streamline seeds describe all the points where streamlines start. Thus, only streamline the go through these points are drawn.
# This may heavily influence the look of the plot!
# In this case only points to the left (*xmin refers to the left, replace with e.g. 0.0 to also obtain streamlines inside the oval)
# of the oval are choosen. Thus, no streamlines inside the oval show.
# In this case, they are equally distributed along the y direction. To also indicate the stagnation streamlines, values slightly above
# and below (+/- 1.e-5) the x axis are added.
radius_for_seeds = 1.0
phi_linspace = np.linspace(0.0, 2*np.pi, 20)
x_seeds = radius_for_seeds * np.sin(phi_linspace)
y_seeds = radius_for_seeds * np.cos(phi_linspace)
streamline_seeds = np.stack([x_seeds, y_seeds], axis=-1)

ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    # minlength=1.0, maxlength=10.0,
    broken_streamlines=False,
    start_points=streamline_seeds,
    color=TUMBlue,
)

# Plot the potential lines.

# The potential line seed describe all points where potential lines start. Here, similar remarks as above apply.
radius_for_seeds = 1.0
phi_linspace = np.linspace(0.0, 2*np.pi, 10)
x_seeds = radius_for_seeds * np.sin(phi_linspace)
y_seeds = radius_for_seeds * np.cos(phi_linspace)
potential_line_seeds = np.stack([x_seeds, y_seeds], axis=-1)

ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    # minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=potential_line_seeds,
    arrowstyle = "-",
    color=TUMOrange,
)

ax.set_aspect('equal', 'box')
plt.show()

Plotting streamlines and potential lines without prescribing any seed, and by that not imposing prior knowledge, is also possible

In [ ]:
plt.close()

fig, ax = plt.subplots()

# Plot the streamlines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    color=TUMBlue,
)

# Plot the potential lines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    arrowstyle = "-",
    color=TUMOrange,
)

ax.set_aspect('equal', 'box')
plt.show()